# Merged

In [ ]:
!pip install ultralytics easyocr opencv-python-headless

import cv2
import os
import numpy as np
import difflib
import csv
import re
from collections import Counter
from ultralytics import YOLO
import easyocr

# ----------------------------
# 📌 Paths
# ----------------------------
VIDEO_PATH = "/content/drive/MyDrive/IOT/License Plate detection model/back/5-3.mp4"
PLATE_MODEL_PATH = "/content/drive/MyDrive/IOT/License Plate detection model/yolov8_license_plate_fast_full_backup/weights/best.pt"
CROPPED_DIR = "/content/drive/MyDrive/IOT/cropped_plates"
ENHANCED_DIR = "/content/drive/MyDrive/IOT/cropped_plates/enhanced_plates"
OUTPUT_CSV = "/content/drive/MyDrive/IOT/plate_results.csv"
FINAL_PLATE_TXT = "/content/drive/MyDrive/IOT/final_plate_number.txt"

os.makedirs(CROPPED_DIR, exist_ok=True)
os.makedirs(ENHANCED_DIR, exist_ok=True)

# ----------------------------
# 🔧 Parameters
# ----------------------------
BIKE_CLASS_ID = 3         # COCO class for motorcycles
BORDER_MARGIN = 5         # Ignore plates near edges
SHARPNESS_THRESHOLD = 100

# ----------------------------
# 🔧 Bengali OCR dictionary
# ----------------------------
areas = [
    "ঢাকা","ঢাকা মেট্রো","টাংগাইল","চট্টগাম","চট্র মেট্রো","খুলনা","খুলনা মেট্রো",
    "বরিশাল","বরিশাল মেট্রো","কক্সবাজার","নেত্রকোণা","রংপুর","রাজ মেট্রো","ভোলা",
    "রাজশাহী","কুষ্টিয়া","নারায়ণগঞ্জ","বগুড়া","সিরাজগঞ্জ","কুমিল্লা","ময়মনসিংহ",
    "ঝিনাইদহ","সিলেট","হবিগঞ্জ","নাটোর","পাবনা","যোশর","বরগুনা","নীলফামারী",
    "পটুয়াখালী","জামালপুর","পিরোজপুর","ব্রাক্ষণবাড়িয়া","মানিকগঞ্জ","নোয়াখালী",
    "বাগেরহাট","সুনামগঞ্জ","চুয়াডাংগা","গোপালগঞ্জ","পঞ্চগড়","লক্ষীপুর","শেরপুর",
    "ঝালকাঠি","খাগড়াছড়ি","কিশোরগঞ্জ","সাতক্ষীরা","নরসিংদী","মৌলভীবাজার","কড়িগ্রাম",
    "শড়িয়তপুর","মাদারীপুর","গাইবান্ধা","রাজবাড়ী","নওয়াবগঞ্জ","রাঙ্গামাটি","চুয়াডাঙ্গা",
    "মুন্সীগঞ্জ","নওগাঁ","গাজীপুর","মেহেরপুর","চাঁপাইনবাবগঞ্জ","বান্দরবান","চাঁদপুর",
    "জয়পুরহাট","নড়াইল","ফরিদপুর","ঠাকুরগাঁও","লালমনিরহাট"
]

vclass = [
    'গ','হ','ল','ঘ','চ','ট','থ','এ',
    'ক','খ','ভ','প','ছ','জ','ঝ','ব',
    'স','ত','দ','ফ','ঠ','ম','ন','অ',
    'ড','উ','ঢ','শ','ই','য','র'
]

dict_words = [f'{area}-{c}' for area in areas for c in vclass]

nums = set('০১২৩৪৫৬৭৮৯')

# ----------------------------
# 🔧 Functions
# ----------------------------
def sharpness_score(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

def enhance_plate(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    sharpened = cv2.filter2D(gray, -1, kernel)
    sharpened = cv2.resize(sharpened, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    _, thresh = cv2.threshold(sharpened, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def extract_license_text(path, reader):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    result = reader.readtext(img, detail=False, paragraph=True)
    area, number = "", ""
    for c in "".join(result)[::-1]:
        if c == "-":
            if len(number) <= 4:
                number += "-"
            else:
                area += "-"
        elif c in nums:
            number += c
        else:
            area += c
    area = area[::-1]
    match = difflib.get_close_matches(area, dict_words, n=1, cutoff=0.5)
    if match:
        area = match[0]
    number = number[::-1]
    if number.find("-") == -1 and len(number) == 6:
        number = number[:2] + "-" + number[2:]
    return area.strip(), number.strip()

def is_valid_plate(area, number):
    pattern = r'^.+-[\u0980-\u09FF]{1} [০-৯]{2}-[০-৯]{4}$'
    combined = f"{area} {number}"
    return re.match(pattern, combined)

# ----------------------------
# 🔧 Load models
# ----------------------------
model_plate = YOLO(PLATE_MODEL_PATH)
model_bike = YOLO("yolov8s.pt")  # COCO pretrained

reader = easyocr.Reader(['bn'], verbose=False, recog_network='bn_license_tps',
                        model_storage_directory="/content/drive/MyDrive/IOT/OCR Models/EasyOCR/models",
                        user_network_directory="/content/drive/MyDrive/IOT/OCR Models/EasyOCR/user_network",
                        download_enabled=False)

# ----------------------------
# 🔧 Process video: crop plates overlapping motorcycles
# ----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    plate_boxes = []
    results_plate = model_plate(frame, conf=0.25)
    for r in results_plate:
        boxes = r.boxes.xyxy.cpu().numpy()
        plate_boxes.extend(boxes)

    bike_boxes = []
    results_bike = model_bike(frame, conf=0.25, classes=[BIKE_CLASS_ID])
    for r in results_bike:
        boxes = r.boxes.xyxy.cpu().numpy()
        bike_boxes.extend(boxes)

    for i, plate in enumerate(plate_boxes):
        px1, py1, px2, py2 = map(int, plate)
        if px1 <= BORDER_MARGIN or py1 <= BORDER_MARGIN or px2 >= frame.shape[1]-BORDER_MARGIN or py2 >= frame.shape[0]-BORDER_MARGIN:
            continue
        overlap = any(not (px2 < bx1 or px1 > bx2 or py2 < by1 or py1 > by2) for bx1, by1, bx2, by2 in bike_boxes)
        if overlap:
            crop = frame[py1:py2, px1:px2]
            if crop.size == 0:
                continue
            cv2.imwrite(os.path.join(CROPPED_DIR, f"frame{frame_id}_plate{i}.jpg"), crop)

    frame_id += 1
cap.release()
print(f"✅ Cropped plates saved in: {CROPPED_DIR}")

# ----------------------------
# 🔧 Enhance cropped plates
# ----------------------------
cropped_files = [f for f in os.listdir(CROPPED_DIR) if f.lower().endswith(('.jpg','.png'))]
for img_file in cropped_files:
    img_path = os.path.join(CROPPED_DIR, img_file)
    img = cv2.imread(img_path)
    if img is None:
        continue
    if sharpness_score(img) < SHARPNESS_THRESHOLD:
        print(f"Skipped blurry image: {img_file}")
        continue
    enhanced = enhance_plate(img)
    cv2.imwrite(os.path.join(ENHANCED_DIR, img_file), enhanced)
print(f"✅ Enhanced plates saved in: {ENHANCED_DIR}")

# ----------------------------
# 🔧 Run OCR on enhanced plates + keep only valid
# ----------------------------
valid_plates = []
enhanced_files = [f for f in os.listdir(ENHANCED_DIR) if f.lower().endswith(('.jpg','.png'))]
for img_file in enhanced_files:
    img_path = os.path.join(ENHANCED_DIR, img_file)
    area, number = extract_license_text(img_path, reader)
    if is_valid_plate(area, number):
        valid_plates.append(f"{area} {number}")
        print(f"VALID: {img_file}: {area} {number}")
    else:
        print(f"INVALID: {img_file}: {area} {number}")

# Save all valid plates to CSV
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["plate"])
    for plate in valid_plates:
        writer.writerow([plate])
print(f"✅ CSV saved at {OUTPUT_CSV}")

# ----------------------------
# 🔧 Majority voting on valid plates
# ----------------------------
if valid_plates:
    most_common_plate = Counter(valid_plates).most_common(1)[0][0]
    with open(FINAL_PLATE_TXT, "w", encoding="utf-8") as f:
        f.write(most_common_plate)
    print(f"✅ Final plate (majority vote) saved in: {FINAL_PLATE_TXT}")
else:
    print("⚠️ No valid plates found for majority voting.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 13.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

0: 320x192 (no detections), 288.3ms
Speed: 13.7ms preprocess, 288.3ms inference, 14.8ms postprocess per image at shape (1, 3, 320, 192)

0: 640x384 (no detections), 466.5ms
Speed: 4.3ms preprocess, 466.5ms inference, 8.1ms postprocess per image at shape (1, 3, 640, 384)

0: 320x192 (no dete

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


INVALID: frame23_plate0.jpg:  ৩১
INVALID: frame24_plate0.jpg: চ ৩-৯৬৩৪
INVALID: frame25_plate0.jpg:  
INVALID: frame27_plate0.jpg:  
INVALID: frame28_plate0.jpg:  
INVALID: frame29_plate0.jpg:  
✅ CSV saved at /content/drive/MyDrive/IOT/plate_results.csv
⚠️ No valid plates found for majority voting.
